# Exploration of Car Make Trends in Singapore 

Data source: [data.gov.sg](https://data.gov.sg/datasets?query=population+by+constituency&page=1&resultId=d_20d3fc7f08caa581c5586df51a8993c5)

Citation:
Land Transport Authority. (2016). Annual Car Population by Make (2024) [Dataset]. data.gov.sg. Retrieved January 11, 2025 from https://data.gov.sg/datasets/d_20d3fc7f08caa581c5586df51a8993c5/view


### Learning Outcomes: 
- Use OpenAPI query to get the latest data instead of downloading and managing `csv` files in data folder.  

In [1]:
import requests
import pandas as pd 
import plotly.express as px

# Sample OpenAPI query from data.gov.sg
dataset_id = "d_20d3fc7f08caa581c5586df51a8993c5"
url = "https://data.gov.sg/api/action/datastore_search?resource_id="  + dataset_id 

In [10]:
# Using API - But records only limited to 100
response = requests.get(url)
data = response.json() 

result = data.get('result', {})
fields = result.get('fields', [])
records = result.get('records', [])

headers = [field['id'] for field in fields]
df = pd.DataFrame(records, columns=headers)
df = df.rename(columns={'_id': 'id'})

print(f"Shape of car make dataframe: {df.shape}")

Shape of car make dataframe: (100, 4)


In [12]:
# Reading full dataset - as at 11 Jan 2025 from csv file saved in data folder 
df = pd.read_csv("../data/AnnualCarPopulationbyMake.csv")
print(f"Shape of car make dataframe: {df.shape}")

Shape of car make dataframe: (1074, 3)


In [17]:
# No. of car make by year
df['year'].value_counts()

year
2005    83
2006    83
2007    83
2008    83
2009    83
2010    83
2011    83
2012    83
2013    83
2014    83
2015    83
2017    81
2016    80
Name: count, dtype: int64

In [19]:
# Filter car makes for each year
car_2015 = set(df[df['year'] == 2015]['make'])
car_2016 = set(df[df['year'] == 2016]['make'])

# Find car makes in 2016 but not in 2015
unique_2015 = car_2015 - car_2016

print("Car makes in 2015 but not in 2016:", unique_2015)

Car makes in 2015 but not in 2016: {'BERTONE', 'SMART', 'LADA', 'MCLAREN', 'MINI COOPER'}


In [18]:
# Filter car makes for each year
car_2016 = set(df[df['year'] == 2016]['make'])
car_2017 = set(df[df['year'] == 2017]['make'])

# Find car makes in 2017 but not in 2016
unique_2017 = car_2017 - car_2016

print("Car makes in 2017 but not in 2016:", unique_2017)

Car makes in 2017 but not in 2016: {'BLUECAR'}


As shown above, the number of car make decreases to 80 in 2016, however, the car make found in 2015 but not 2016 include BERTONE, SMART, LADA, MCLAREN, MINI COOPER. In 2017, one additional car make (BLUECAR) is observed.

In [15]:
# Number of unique car make in the dataset
df['make'].nunique()

86

Given that there are **86** unique car make, it might be difficult to visualise everything into a single chart. Hence, it might be useful to focus on potentially interesting data points such as the most common and most uncommon car make, and any increasing or decreasing trend observed over the years. 

In [13]:
# Compute average number by car make over the years (2005 - 2017)
df.head()

,year,make,number
0,2005,ALFA ROMEO,914
1,2005,ALPINA,0
2,2005,ASTON MARTIN,23
3,2005,AUDI,2025
4,2005,AUSTIN,137


In [41]:
# Finding the most popular car make in SG over the years (2005 - 2017)
most_common_make = df.groupby(['make'])[['number']].mean().round().astype(int).reset_index().sort_values('number', ascending=False)
most_common_make.head(5)

,make,number
80,TOYOTA,149391
30,HONDA,81877
59,NISSAN,55615
32,HYUNDAI,40144
48,MERCEDES BENZ,35157


In [40]:
most_common_make.tail(5)

,make,number
45,MAYBACH,1
74,SMART,0
38,LADA,0
51,MINI COOPER,0
7,BERTONE,0


Top 5 most common car make in Singapores are: Toyota, Honda, Nissan, Hyundai, and Mercedes Benz. 
Bottom 5 car make in Singapores are: MayBach, Smart, Lada, Mini cooper, Bertone.

Based on intuition, there could be prolems with the data collected given that mini cooper are common in Singapore roads.
Data came from LTA and figures include cars exempted from tax and vehicle quota system. However, it's unclear about the time period of data collection. 

Does the car make only include newly and renewed issued COE cars for that year or all cars in Singapore? Perhaps, there are missing or unaccounted data that were not included in the database. 

In [42]:
top_10_make = list(most_common_make.head(10)['make'].unique())
filtered_top10 = df[df['make'].isin(top_10_make)]
filtered_top10['est_number'] = filtered_top10['number'].round(-3).astype('int') 

fig = px.line(
    filtered_top10,
    x = 'year', 
    y = 'est_number',
    color = 'make',
    title = 'Top 10 Car Makes in Singapore (2005 - 2017)', 
    labels={'est_number': 'Count', 'year': 'Year', 'make': 'Car Model'}, 
    template='plotly_white', 
    markers=True)

fig.show()

/var/folders/b3/3sxmjwh95_n18bf__pwfsbnw0000gn/T/ipykernel_10220/3235125736.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



By plotting the line chart, we observe the trends of the popular car make in Singapore. 

**Toyota**
- Steadily decreasing since 2010. 
- Perceived as [one of the most reliable Japanese brand](https://kaizenaire.com/sg/top-10-most-reliable-car-brands-for-singaporean-drivers/#:~:text=According%20to%20their%20latest%20report,Porsche%2C%20BMW%2C%20and%20Kia.) in Singapore.
- Given the high cost of car ownership in Singapore, it's natural for consumers to seek brands that are reknown for it's reliability and low maintenance cost for the next 10 (or less) years before the certificate of entitlement (COE) expires. 
- With that in mind, demand for Toyota and Honda are the highest in Singapore. 
- But supplier could leverage on this competitve advantage to charge a premium in Singapore. Hence, the cost of a Toyota car have been steadily increasing over the years. The price of Toyota brand new cars come close to Mercedes Benz in recent years. But driving experience of continental car still trumps over commercial Japanese car.

**Honda**
- Observed a spike from around 48k in 2005 to 86k in 2008. Another increase observed from 86k in 2015 to 94k in 2016. 
- One possible reason for the increase in recent years may be a result from public driving center for learners use Honda jazz as their training cars. 
- Honda, similar to Toyota, is known for its reliability, low maintenance and value for money. 
- It's perceived as one of the safer and value for money option for first time owners, small families, and middle-income individuals. 

**Nissan**
- Used to be one of the top 3 most common car make in Singapore in the early years 2005-2006. However, the number decreases from 65k in 2005 to 40k in 2017 (~23% decline). 
- In recent years, it was known that Nissan has been facing financial woes and had to merge with Renault in 1999. There are talks about alliance among Renault, Nissan and Mitsubishi targeted in 2030. [Carlos Ghosn](https://www.drive.com.au/news/is-nissan-really-in-serious-trouble/) who ran both Renault and Nissan during merger fled Japan in 2018 after facing arrest for allegedly misusing company funds. Sales of Nissan cars have been declining especially in one of the most important market - China resulting in 9,000 jobs cut and reduced ownership stake in Mitsubishi. 
- Additionally, Nissan cars have been perceived as increasingly unreliable and appearing less on searches of reliable car brands. From auto-dealers to car instructors, the bad reviews of Nissan car especially in terms of reliability have been popping up. 

**Mercedes Benz**
- One of the luxury car brand in Singapore but was among the top 3 from 2016 to 2017. Numbers have been steadily increasing over the years. 
- Mercedes Benz is perceived as one of the most reliable to own continental cars in Singapore. It's also the go-to option for high income earners as a head-turner and a symbol of wealth and status. 
- Both interior and exterior of the vehicles used high quality materials instead of plastics found in most Japanese and Korean car makers. 
- Driving experience of the Mercedes Benz awed many car lovers. Moreover, it's much more dependable and reliable when compared to other continental luxury car brands. 

**BMW** 
- Another luxury car brand that many individual in Singapore loved. It's among the top 5 in recent years with numbers showing an increasing trend throughout the years. 
- Reasons for the increase is quite similar to Mercedes Benz, with the major reasoning for better driving experience. It's also a symbol of wealth and status among the higher income earners. It's not rare to hear conversations around cars that "Mercedes Benz and BMW are bosses' car brands".

##### Other noteworthy brands

**Mazda**
- Increasing trend observed from 20k in 2005 to 32k in 2017 (~60% increase). 
- Mazda is increasing popular in Singapore given it's perceived reliability, better driving experience, comparable interior fittings to continental cars and value for money. 

**Volkswagon**
- Increasing trend observed from 3k in 2005 to 28k.
- Mainly due to the lower price and higher PARF value of Volkswagon cars, resulting in lower depreciation. Hence, for cost-consious buyers who looks for continental cars driving experience, Volkswagon becomes more of the more viable choice in recent years.

In [45]:
df['make'].unique()

array(['ALFA ROMEO', 'ALPINA', 'ASTON MARTIN', 'AUDI', 'AUSTIN', 'B.M.W.',
       'BENTLEY', 'BERTONE', 'CADILLAC', 'CHANA', 'CHERY', 'CHEVROLET',
       'CHRYSLER', 'CITROEN', 'DAEWOO', 'DAIHATSU', 'DAIMLER', 'DATSUN',
       'DODGE', 'DONGFENG', 'FERRARI', 'FIAT', 'FORD', 'FOTON', 'GEELY',
       'GOLDEN DRAGON', 'HAFEI', 'HOLDEN', 'HONDA', 'HUMMER', 'HYUNDAI',
       'INFINITI', 'ISUZU', 'JAGUAR', 'JEEP', 'KIA', 'LADA',
       'LAMBORGHINI', 'LANCIA', 'LAND ROVER', 'LOTUS', 'M.G.', 'MASERATI',
       'MAYBACH', 'MAZDA', 'MCLAREN', 'MERCEDES BENZ', 'MG-F', 'MINI',
       'MINI COOPER', 'MINI MAYFAIR', 'MITSUBISHI', 'MITSUOKA', 'MORGAN',
       'MORRIS', 'NAZA', 'NISSAN', 'OPEL', 'OTHERS', 'PANTHER', 'PERODUA',
       'PEUGEOT', 'PORSCHE', 'PROTON', 'RENAULT', 'ROLLS ROYCE', 'ROVER',
       'SAAB', 'SEAT', 'SINGER', 'SKODA', 'SMART', 'SSANGYONG', 'SUBARU',
       'SUNBEAM', 'SUZUKI', 'TD CARS', 'TOYOTA', 'TRIUMPH', 'VOLKSWAGEN',
       'VOLVO', 'WULING', 'ZOTYE', 'BYD', 'Mclaren', 'BL

In [80]:
make = 'VOLVO'
filtered_make = df[df['make'] == make.upper()]

fig = px.line(
    filtered_make,
    x = 'year', 
    y = 'number',
    color = 'make',
    title = f'Car Make: {make.upper()} in Singapore (2005 - 2017)', 
    labels={'number': 'Count', 'year': 'Year', 'make': 'Car Model'}, 
    template='plotly_white', 
    markers=True)

fig.show()

Code above allows exploration of individual car make trends. However, more recent car models such as Tesla, Polestar may not be available in the existing data. 

**PORSCHE**
- Have been increasing over the years (i.e., straight line) - from 763 in 2005 to 5339 in 2017. 
- Easily one of the most expensive car brands to own in Singapore.
- When looking at the millionaire density and the porsche car ownership side by side, there seem to be a correlation between the two.

**MASERATI**
- Steadily increasing over the years. With a showroom opening in 2017 as demand grows, it's one of the cheaper alternative among luxury sports car brands such as Porsche, Ferrari. 

**AUDI**
- Another continental car brand that is popular among the high-middle to high income earners is Audi. A car that prides itself in terms of design and build quality. 
- It has a similar status close to Mercedes Benz and BMW in Singapore. 
- Additionally, Audi has added a new shownroom near Chinatown in 2024 allowing a direct customer service to buyers. The maker is closely linked to popular German car makes such as Porsche and Volkswagon in Singapore. 